In [ ]:
import torch

# Status quo

In [ ]:
def series_to_tensors(series, lookaheadSize=5):

    X,y = [],[]
    for i in np.arange(5,len(series)-1):
        X.append(series[i-lookaheadSize:i])
        y.append(series[i+1])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(len(series)-lookaheadSize-1,1,5)
    y=y.reshape(-1,1)

    dataset = torch.utils.data.TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
    
    return dataset, torch.from_numpy(X), y

In [ ]:
def dataprep(args):

    stockData = pd.read_csv(args.data, index_col='Date')
    stock_train_df, stock_test_df = train_test_split(stockData, test_size=args.test_train_ratio)

    print(stock_train_df)

    # Instead of this use LayerNorm or BatchNorm in the neural net
    scaler = MinMaxScaler().fit(stock_train_df)
    stock_train_df = scaler.transform(stock_train_df)
    stock_test_df = scaler.transform(stock_test_df)

    train_tensors,_,_ = series_to_tensors(stock_train_df)
    _,X_test,y_test = series_to_tensors(stock_test_df)

    return scaler, train_tensors, X_test, y_test

In [ ]:
class lstm_model(torch.nn.Module):

    def __init__(self):
        super(lstm_model, self).__init__()
        self.lstm1=torch.nn.LSTM(batch_first=True, input_size=5, hidden_size=1)
        self.out=torch.nn.Linear(1,1)

    def forward(self, x, hidden=None):
        x, hidden = self.lstm1(x)
        x = x[:,-1]
        x = self.out(x)
        return x, hidden

In [ ]:
def train(trainset, epochs):

    seq_model = lstm_model()
    optim = torch.optim.Adam(lr = 0.0001, params=seq_model.parameters())

    for epoch in np.arange(epochs):

        Loss=0

        for data in trainset:

            feats, target = data
            optim.zero_grad()

            y_p,_ = seq_model(feats.float())
            loss = torch.nn.functional.mse_loss(y_p.float(), target.float())

            loss.backward()
            optim.step()
            Loss += loss.item()

        mlflow.log_metric("Training loss", Loss, step=epoch)
    return seq_model


In [ ]:
# Load Scaler object later and send it for scaling data

scaler, trainset, X_test, y_test = dataprep(args)

epochs=10
trainedModel = train(trainset, epochs)

trainedModel.eval()

y_pred,_=trainedModel(X_test.float())

mape=mean_absolute_percentage_error(y_test, y_pred.detach().numpy())

# Improved code

In [1]:
import pytorch_lightning as pl
import numpy as np 
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

In [2]:
class dataset(pl.LightningDataModule):

    def __init__(self, data=None, scaler=None):
        super(dataset,self).__init__()
        self.lookback_size = 5
        self.batch_size = 32

        if data is not None:
            self.data=data
            self.series = pd.read_csv(self.data, index_col='Date')
            # Train:valid:test = 80:10:10
            self.train_df, self.valid_df = train_test_split(self.series, test_size=0.2)
            self.valid_df, self.test_df = train_test_split(self.valid_df, test_size=0.5)

        if scaler is None and data is not None:
            self.scaler = MinMaxScaler().fit(self.train_df)
        else:
            self.scaler=scaler

    def train_tensors(self,df):

        X, y = [], []

        for i in np.arange(self.lookback_size, len(df)-1):
            X.append(df[i-self.lookback_size:i])
            y.append(df[i+1])

        X = np.array(X).reshape(-1,self.lookback_size,1)
        y = np.array(y).reshape(-1,1)
        return TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
    
    def setup(self, stage=None):
        
        self.train_df = self.scaler.transform(self.train_df) 
        self.valid_df = self.scaler.transform(self.valid_df) 
        self.test_df = self.scaler.transform(self.test_df) 

        self.train_data = self.train_tensors(self.train_df)
        self.valid_data = self.train_tensors(self.valid_df)
        self.test_data = self.train_tensors(self.test_df)

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size)

    def val_dataloader(self):
       return DataLoader(self.valid_data, batch_size=self.batch_size)

    def test_dataloader(self):
       return DataLoader(self.test_data, batch_size=self.batch_size)

In [3]:
class model(pl.LightningModule):

    def __init__(self,lookback_size=5):

        super(model,self).__init__()

        self.lookback_size = lookback_size
        self.lstm=torch.nn.LSTM(batch_first=True, input_size=1, hidden_size=self.lookback_size)
        self.out=torch.nn.Linear(5,1)
        self.loss=torch.nn.functional.mse_loss

    def forward(self, x, hidden=None):
        x, hidden = self.lstm(x)
        x = x[:,-1]
        x = self.out(x)
        return x, hidden

    def configure_optimizers(self):
        return torch.optim.Adam(params=self.parameters(), lr=1e-3)

    def configure_callbacks(self):
        early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=5)
        checkpoint = ModelCheckpoint(monitor="val_loss")
        return [early_stop, checkpoint]
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch 
        logits,_ = self.forward(x.type(torch.float32)) 
        loss = self.loss(logits.float(), y.float()) 
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, valid_batch, batch_idx): 
        x, y = valid_batch 
        logits,_ = self.forward(x.type(torch.float32)) 
        loss = self.loss(logits.float(), y.float())
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    def test_step(self, test_batch, batch_idx): 
        x, y = test_batch 
        logits,_ = self.forward(x.type(torch.float32)) 
        loss = self.loss(logits.float(), y.float())
        self.log("test_loss", loss)

In [4]:
def train():

    trainer=pl.Trainer(max_epochs=5)
    datamod=dataset('../data/WIPRO.NS.csv')
    mod=model()
    trainer.fit(model=mod, datamodule=datamod)
    trainer.test(model=mod, datamodule=datamod)
    return mod, datamod.scaler


In [5]:
model,scalerObj=train()

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/azureML/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniforge/base/envs/azureML/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cac

Sanity Checking: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/azureML/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniforge/base/envs/azureML/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniforge/base/envs/azureML/lib/python3.11/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: EarlyStopping, ModelCheckpoint
/opt/homebrew/Caskroom/miniforge/base/envs/azureML/lib/python3.11/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/azureML/lib/python3.11/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/azureML/lib/python3.11/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/azureML/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUse

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           1.4024996757507324
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [6]:
import pickle

In [7]:
pickle.dump(scalerObj, open('./outputs/scaler.pkl','wb'))
model_file = f"./outputs/model.pth"
torch.save(model.state_dict(), model_file)

Restart notebook and then run the below cells

In [7]:
modex=model()
modex.load_state_dict(torch.load('./outputs/model.pth'))
modex.eval()
with open('./outputs/scaler.pkl','rb') as f:
    scaler=pickle.load(f)
datamodex=dataset(scaler=scaler)

In [8]:
pred_data=datamodex.predict_dataloader(data=pd.DataFrame({"Close":{"d1":2663,"d2":2654.4,"d3":2698,"d4":2690,"d5":2698.12}}))

In [11]:
x,_=modex(pred_data)

In [12]:
scaler.inverse_transform(x.detach().numpy())

array([[393.27002]], dtype=float32)